<a href="https://colab.research.google.com/github/pawel0508/MachineLearning_UcznieNienadzorowane/blob/main/Association2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Przyklad apriori 

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.float_format', lambda x: f'{x:.2f}')

In [4]:
!wget https://storage.googleapis.com/esmartdata-courses-files/ml-course/products.csv
!wget https://storage.googleapis.com/esmartdata-courses-files/ml-course/orders.csv

--2020-10-09 21:10:20--  https://storage.googleapis.com/esmartdata-courses-files/ml-course/products.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.210.128, 173.194.214.128, 173.194.216.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.210.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2166953 (2.1M) [application/octet-stream]
Saving to: ‘products.csv.1’

products.csv.1      100%[===================>]   2.07M  --.-KB/s    in 0.01s   

2020-10-09 21:10:21 (165 MB/s) - ‘products.csv.1’ saved [2166953/2166953]

--2020-10-09 21:10:21--  https://storage.googleapis.com/esmartdata-courses-files/ml-course/orders.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.204.128, 172.217.203.128, 74.125.139.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24680147 (24M) [application/octe

In [3]:
products = pd.read_csv('products.csv', usecols=['product_id', 'product_name'])
products.head()

,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce


In [4]:
orders = pd.read_csv('orders.csv', usecols=['order_id', 'product_id'])
orders.head()

,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633


Przygotowanie danych

In [5]:
data = pd.merge(orders, products, how = 'inner', on = 'product_id', sort = True)
data = data.sort_values(by = 'order_id')
data.head()

,order_id,product_id,product_name
588447,1,22035,Organic Whole String Cheese
256931,1,10246,Organic Celery Hearts
277441,1,11109,Organic 4% Milk Fat Whole Milk Cottage Cheese
1194893,1,43633,Lightly Smoked Sardines in Olive Oil
1302365,1,47209,Organic Hass Avocado


In [6]:
data.describe()

,order_id,product_id
count,1384617.00,1384617.00
mean,1706297.62,25556.24
std,989732.65,14121.27
min,1.00,1.00
25%,843370.00,13380.00
50%,1701880.00,25298.00
75%,2568023.00,37940.00
max,3421070.00,49688.00


In [7]:
data['product_name'].value_counts()

Banana                                               18726
Bag of Organic Bananas                               15480
Organic Strawberries                                 10894
Organic Baby Spinach                                  9784
Large Lemon                                           8135
                                                     ...  
Light Thousand Island Dressing                           1
Restaurante Style Original Premium Tortilla Chips        1
Lamb Chops                                               1
Artichoke And Feta Ravioli                               1
Mountain Spring Scent Liquid Fabric Softener             1
Name: product_name, Length: 39123, dtype: int64

In [8]:
data['order_id'].nunique()

131209

In [9]:
transactions = data.groupby(by = 'order_id')['product_name'].apply(lambda x: ','.join(x))

In [10]:
transactions = transactions.str.split(',')
transactions

order_id
1          [Organic Whole String Cheese, Organic Celery H...
36         [Super Greens Salad, Spring Water, Organic Gar...
38         [Green Peas, Organic Baby Arugula, Shelled Pis...
96         [Organic Raspberries, Organic Pomegranate Kern...
98         [Uncured Applewood Smoked Bacon, Black Beans, ...
                                 ...                        
3421049    [Gluten Free Rice Bread, Organic Baby Broccoli...
3421056    [Sparkling Lemon Water, Brioche Buns, Homestyl...
3421058    [Classic Britannia Crisps, Club Soda Lower Sod...
3421063    [Natural Artesian Water, Organic Half & Half, ...
3421070    [Broccoli Florettes, Organic Unsweetened Almon...
Name: product_name, Length: 131209, dtype: object

Kodowanie transakcji

In [14]:
from mlxtend.preprocessing import TransactionEncoder
encoder = TransactionEncoder()
encoder.fit(transactions)
transactions_encoded = encoder.transform(transactions, sparse = True)
transactions_encoded

<131209x40434 sparse matrix of type '<class 'numpy.bool_'>'
	with 1442410 stored elements in Compressed Sparse Row format>

In [15]:
transactions_encoded_df = pd.DataFrame(transactions_encoded.toarray(), columns = encoder.columns_)
transactions_encoded_df

,,Apricot & Banana Stage 2 Baby Food,Broad Spectrum SPF 30,Instant,Livermore Valley,Low Sodium Marinara,Premium,Vetiver scent,Whole,#2,& Baby Wipes,& Blueberry with Quinoa Organic Baby Food,& Cheese Biscuit,& Cheese Croissant,& Cheese English Muffin,& Cheese Sandwiches,& Cheese Sauce,& Cheese Scramble,& Coconut With Red Lentils Organic Baby Food,& Garlic Pasta Sauce,& Grape Ice Pops,& Kale Ravioli,& Peas,& Pineapple,& Raisin,& Seasoning,& Sliced,& Yellow Peppers,& orange,0,0.87 oz) Air Care,004 Dark Brown,1,1 Ply,1 Subject,1 mg,1 to 1,1% Lowfat,1% Milkfat,1-1/2% Milkfat,...,for Tots Apple White Grape Juice,for Women Maximum Absorbency L Underwear,from Concentrate Mango Nectar,fruitwater® Strawberry Kiwi Sparkling Water,gel hand wash sea minerals,gelato Coffee Toffee,go fresh Cool Moisture Beauty,iChef Casserole Pans with Lids (10 7/16 in x 8 in x 1 3/4 in),in 100% Juice Mixed Fruit,in Gravy with Carrots Peas & Corn Mashed Potatoes & Meatloaf Nuggets,kattle Boiled & Hearth Baked Slice,o.b Super Plus Fluid Lock Tampons,of Hanover 100 Calorie Pretzels Mini,of Norwich Original English Mustard Powder Double Superfine,pumpkin spice,rich kiss Olive & Aloe Moisturizer 2 in 1,smart Blend Chicken & Rice Formula Dry Dog Food,smartwater® Electrolyte Enhanced Water,vitaminwater® XXX Acai Blueberry Pomegranate,w/Banana Pulp Free Juice,with Bleach Disinfectant Cleanser Scratch Free Lavender Fresh,with Bleach Powder Cleanser,with Crispy Almonds Cereal,with Dawn Action Pacs Fresh Scent Dishwasher Detergent Pacs,with Olive Oil Mayonnaise,with Olive Oil Mayonnaise Dressing,with Pump Rebalancing Shampoo,with Seasoned Roasted Potatoes Scrambled Eggs & Sausage,with Sweet & Smoky BBQ Sauce Cheeseburger Sliders,with Sweet Cinnamon Bunches Cereal,with Xylitol Cinnamon 18 Sticks Sugar Free Gum,with Xylitol Island Berry Lime 18 Sticks Sugar Free Gum,with Xylitol Minty Sweet Twist 18 Sticks Sugar Free Gum,with Xylitol Original Flavor 18 Sticks Sugar Free Gum,with Xylitol Unwrapped Original Flavor 50 Sticks Sugar Free Gum,with Xylitol Unwrapped Spearmint 50 Sticks Sugar Free Gum,with Xylitol Watermelon Twist 18 Sticks Sugar Free Gum,with a Splash of Mango Coconut Water,with a Splash of Pineapple Coconut Water,Lightly Seasoned with Rosemary and Roasted Garlic Family Size Herb Chicken Tortellini
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,

Algorytm apriori

In [17]:
from mlxtend.frequent_patterns import apriori, association_rules

supports = apriori(transactions_encoded_df.iloc[:1000,:], min_support = 0.01, 
                   use_colnames = True, n_jobs = 1)
supports = supports.sort_values(by = 'support', ascending = False)
supports.head(10)

,support,itemsets
11,0.13,(Banana)
94,0.10,(Organic Strawberries)
10,0.10,(Bag of Organic Bananas)
49,0.07,(Organic Baby Spinach)
37,0.07,(Large Lemon)
71,0.06,(Organic Hass Avocado)
58,0.05,(Organic Cucumber)
40,0.04,(Limes)
45,0.04,(Organic Avocado)
122,0.04,(Strawberries)


In [18]:
rules = association_rules(supports, metric='confidence', min_threshold=0)
rules = rules.iloc[:, [0, 1, 4, 5, 6]]
rules = rules.sort_values(by='lift', ascending=False)
rules.head(15)

,antecedents,consequents,support,confidence,lift
48,( Bunch),(Organic Red Radish),0.01,0.61,55.56
49,(Organic Red Radish),( Bunch),0.01,1.00,55.56
64,(Limes),(Organic Cilantro),0.01,0.22,7.41
65,(Organic Cilantro),(Limes),0.01,0.33,7.41
38,(Organic Garnet Sweet Potato (Yam)),(Organic Baby Spinach),0.01,0.48,6.55
39,(Organic Baby Spinach),(Organic Garnet Sweet Potato (Yam)),0.01,0.15,6.55
67,(Limes),(Organic Avocado),0.01,0.22,4.94
66,(Organic Avocado),(Limes),0.01,0.22,4.94
59,(Organic Baby Spinach),(Organic Red Onion),0.01,0.14,4.28
58,(Organic Red Onion),(Organic Baby Spinach),0.01,0.31,4.28
